In [3]:
! pip install -r requirements.txt --quiet

In [ ]:
from semantic_kernel import Kernel
from user_plugins import WeatherPlugin, DataCenterPlugin
from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from dotenv import load_dotenv
from os import environ

load_dotenv(override=True)

kernel = Kernel()

kernel.add_service(AzureChatCompletion(
    service_id="chat",
    deployment_name=environ["AZURE_OPENAI_MODEL"],
    endpoint=environ["AZURE_OPENAI_ENDPOINT"],
    api_key=environ["AZURE_OPENAI_API_KEY"] ))

kernel.add_plugin(WeatherPlugin, plugin_name="Weather")
kernel.add_plugin(DataCenterPlugin, plugin_name="DataCenter")

agent_instructions = (
    "You are an intelligent assistant. You have access to two plugins: "
    "the Weather plugin and the DataCenter plugin. "
    "Use the Weather plugin to answer questions about weather forecasts, temperatures, "
    "and conditions. Always include temperatures in both Celsius and Fahrenheit. "
    "Use the DataCenter plugin to assist with data center-related queries, such as "
    "energy usage, performance, or any related data center information. "
    "Be concise, friendly, and helpful. If you're unsure about a location or data center, ask for clarification."
)

# Create the agent
agent = ChatCompletionAgent(
    kernel=kernel, 
    name="DataCenterAgent", 
    instructions=agent_instructions
)

In [2]:
response = await agent.get_response(messages="Describe the resource intensity of data center facility infrastructure")
print(response)

The resource intensity of data center facility infrastructure is commonly described using two key metrics:

1. **Power Usage Effectiveness (PUE)**: PUE measures the total energy consumed by a data center in relation to the energy used by its IT equipment. It is a dimensionless metric (kWh/kWh) highlighting the efficiency of the facility, with lower values indicating more efficient infrastructure.

2. **Water Usage Effectiveness (WUE)**: This metric quantifies the total water consumption by the facility per unit of energy consumed by IT equipment, typically measured in liters per kWh. On-site water usage (WUE [site]) is often linked to cooling infrastructure, while water usage tied to electricity generation (WUE [source]) deals with the water intensity of energy generation.

Variability in PUE and WUE depends on cooling systems, design practices, operational systems, and geographical conditions. Efficient utilization of resources through optimized cooling system design, siting, and oper

## [Handling Intermediate Messages with a ChatCompletionAgent](https://learn.microsoft.com/en-us/semantic-kernel/frameworks/agent/chat-completion-agent?pivots=programming-language-python#handling-intermediate-messages-with-a-chatcompletionagent)

The Semantic Kernel ChatCompletionAgent is designed to invoke an agent that fulfills user queries or questions. During invocation, the agent may execute tools to derive the final answer. To access intermediate messages produced during this process, callers can supply a callback function that handles instances of FunctionCallContent or FunctionResultContent.



In [3]:

from semantic_kernel.contents import AuthorRole, ChatMessageContent, FunctionCallContent, FunctionResultContent


# Define a list to hold callback message content
intermediate_steps: list[ChatMessageContent] = []

# Define an async method to handle the `on_intermediate_message` callback
async def handle_intermediate_steps(message: ChatMessageContent) -> None:
    intermediate_steps.append(message)



user_inputs = [
    "Hello", 
    "What's the weather like in sf today?", 
    "Describe the resource intensity of data center facility infrastructure"
    "What time is the sunrise?",
    "Thank you",
]

thread = None

# Generate the agent response(s)
for user_input in user_inputs:
    print(f"# {AuthorRole.USER}: '{user_input}'")
    async for response in agent.invoke(
        messages=user_input,
        thread=thread,
        on_intermediate_message=handle_intermediate_steps,
    ):
        thread = response.thread
        print(f"# {response.name}: {response.content}")

# Delete the thread when it is no longer needed
await thread.delete() if thread else None

# Print the intermediate steps
print("\nIntermediate Steps:")
for msg in intermediate_steps:
    if any(isinstance(item, FunctionResultContent) for item in msg.items):
        for fr in msg.items:
            if isinstance(fr, FunctionResultContent):
                print(f"Function Result:> {fr.result} for function: {fr.name}")
    elif any(isinstance(item, FunctionCallContent) for item in msg.items):
        for fcc in msg.items:
            if isinstance(fcc, FunctionCallContent):
                print(f"Function Call:> {fcc.name} with arguments: {fcc.arguments}")
    else:
        print(f"{msg.role}: {msg.content}")

# AuthorRole.USER: 'Hello'
# DataCenterAgent: Hi there! How can I assist you today?
# AuthorRole.USER: 'What's the weather like in sf today?'
# DataCenterAgent: The weather in San Francisco today is foggy with a temperature of 60°F (about 15.6°C). Let me know if you'd like additional details!
# AuthorRole.USER: 'Describe the resource intensity of data center facility infrastructureWhat time is the sunrise?'
# DataCenterAgent: ### Resource Intensity of Data Center Infrastructure
Data centers measure their resource intensity using metrics like **Power Usage Effectiveness (PUE)** and **Water Usage Effectiveness (WUE)**. 
- **PUE** is the ratio of total data center electricity demand to IT equipment electricity demand. A lower PUE indicates higher energy efficiency.
- **WUE** measures water consumption per kWh and is primarily impacted by cooling systems, operations, and local climate. Cooling infrastructure and operational practices significantly influence water use and energy performance